In [ ]:
from pathlib import Path
import nbformat
from nbformat import v4, write
import os

# Full Python script content as a list of cells
cells = []

# Title cell (Markdown)
cells.append(v4.new_markdown_cell("""\
# Claims Dashboard - Dash Application

This interactive dashboard visualizes insurance claims data with filtering capabilities.
"""))

# Code cell: Install dependencies (commented out for Render deployment)
cells.append(v4.new_code_cell("""\
# # Install necessary packages (uncomment for Colab/Jupyter)
# !pip install dash pandas plotly flask-ngrok flask-caching openpyxl gunicorn
"""))

# Code cell: Configuration and imports
cells.append(v4.new_code_cell("""\
import os
import pandas as pd
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
from flask import Flask
from flask_caching import Cache

# Configuration
DEBUG = os.environ.get('DEBUG', 'False').lower() in ('true', '1', 't')
PORT = int(os.environ.get('PORT', 8050))

# Try to import Colab-specific modules
try:
    from flask_ngrok import run_with_ngrok
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Data file handling
if IN_COLAB:
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/MyDrive/Excel2Dashboard.xlsx'
    except:
        file_path = 'Excel2Dashboard.xlsx'
else:
    file_path = 'data/Excel2Dashboard.xlsx'  # For local/Render deployment
"""))

# Code cell: Initialize app
cells.append(v4.new_code_cell("""\
# Initialize Flask and Dash
server = Flask(__name__)
app = dash.Dash(__name__, server=server,
                meta_tags=[{'name': 'viewport',
                           'content': 'width=device-width, initial-scale=1.0'}])

# Configure caching
cache = Cache(app.server, config={
    'CACHE_TYPE': 'SimpleCache',
    'CACHE_DEFAULT_TIMEOUT': 300  # 5 minutes
})

# Enable ngrok if in Colab
if IN_COLAB:
    run_with_ngrok(server)
"""))

# Code cell: Data loading and preprocessing
cells.append(v4.new_code_cell("""\
# Load or create sample data
try:
    df = pd.read_excel(file_path)
except Exception as e:
    print(f"Error loading data file: {e}. Creating sample data...")
    data = {
        'Date': pd.date_range(start='2023-01-01', periods=100, freq='D').tolist(),
        'Claim_Stage': ['Submitted']*40 + ['Approved']*35 + ['Rejected']*25,
        'Insurer': ['Aetna']*40 + ['BlueCross']*35 + ['United']*25,
        'Claim_Type': ['Medical']*50 + ['Dental']*30 + ['Vision']*20,
        'Claim_Amount': [round(x, 2) for x in
                         list(np.random.normal(5000, 1500, 50)) +
                         list(np.random.normal(3000, 800, 30)) +
                         list(np.random.normal(2000, 500, 20))],
        'Batch_Amount': [round(x, 2) for x in
                         list(np.random.normal(5500, 1600, 50)) +
                         list(np.random.normal(3200, 900, 30)) +
                         list(np.random.normal(2100, 600, 20))]
    }
    df = pd.DataFrame(data)
    os.makedirs('data', exist_ok=True)
    df.to_excel('data/Excel2Dashboard.xlsx', index=False)

# Clean and preprocess data
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()
df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
df['claim_amount'] = pd.to_numeric(df['claim_amount'], errors='coerce')
df['batch_amount'] = pd.to_numeric(df['batch_amount'], errors='coerce')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month_name()
df['quarter'] = df['date'].dt.quarter

# Create some derived metrics
df['claim_status'] = np.where(df['claim_amount'] > df['batch_amount'],
                             'Over',
                             np.where(df['claim_amount'] < df['batch_amount'],
                                     'Under', 'Exact'))
"""))

# Code cell: App layout
cells.append(v4.new_code_cell("""\
# Define app layout
app.layout = html.Div([
    html.Div([
        html.H1("Claims Dashboard", className="header-title"),
        html.P("Explore and filter insurance claims data", className="header-description"),
    ], className="header"),

    html.Div([
        html.Div([
            html.Label("Claim Stage", className="dropdown-label"),
            dcc.Dropdown(
                id='stage-dropdown',
                options=[{'label': val, 'value': val} for val in df['claim_stage'].unique()],
                value='Submitted',
                clearable=False,
                className="dropdown"
            ),
        ], className="dropdown-container"),

        html.Div([
            html.Label("Insurer", className="dropdown-label"),
            dcc.Dropdown(
                id='insurer-dropdown',
                options=[{'label': 'All Insurers', 'value': 'All'}] +
                        [{'label': val, 'value': val} for val in df['insurer'].unique()],
                value='All',
                clearable=False,
                className="dropdown"
            ),
        ], className="dropdown-container"),

        html.Div([
            html.Label("Claim Type", className="dropdown-label"),
            dcc.Dropdown(
                id='type-dropdown',
                options=[{'label': 'All Types', 'value': 'All'}] +
                        [{'label': val, 'value': val} for val in df['claim_type'].unique()],
                value='All',
                clearable=False,
                className="dropdown"
            ),
        ], className="dropdown-container"),

        html.Div([
            html.Label("Year", className="dropdown-label"),
            dcc.Dropdown(
                id='year-dropdown',
                options=[{'label': 'All Years', 'value': 'All'}] +
                        [{'label': val, 'value': val} for val in sorted(df['year'].unique())],
                value='All',
                clearable=False,
                className="dropdown"
            ),
        ], className="dropdown-container"),
    ], className="filters-row"),

    html.Div([
        html.Div([
            html.Div(id='total-claims', className="metric-card"),
            html.Div(id='total-amount', className="metric-card"),
            html.Div(id='avg-amount', className="metric-card"),
        ], className="metrics-container"),
    ]),

    html.Div([
        dcc.Graph(id='claims-by-year', className="graph"),
        dcc.Graph(id='amount-distribution', className="graph"),
    ], className="graphs-row"),

    html.Div([
        dcc.Graph(id='claims-by-insurer', className="graph"),
        dcc.Graph(id='claims-by-type', className="graph"),
    ], className="graphs-row"),

    html.H3("Filtered Claims Data", className="table-header"),
    html.Div(id='data-table', className="table-container"),

    dcc.Store(id='filtered-data-store'),
], className="container")
"""))

# Code cell: CSS styling
cells.append(v4.new_code_cell("""\
# CSS styling
app.css.append_css({
    'external_url': 'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css'
})

app.css.append_css({
    'external_url': 'https://fonts.googleapis.com/css2?family=Open+Sans:wght@400;600&display=swap'
})

styles = """
.container {
    font-family: 'Open Sans', sans-serif;
    max-width: 1200px;
    margin: 0 auto;
    padding: 20px;
}

.header {
    text-align: center;
    margin-bottom: 30px;
}

.header-title {
    color: #2c3e50;
    margin-bottom: 10px;
}

.header-description {
    color: #7f8c8d;
}

.filters-row {
    display: flex;
    flex-wrap: wrap;
    gap: 20px;
    margin-bottom: 20px;
}

.dropdown-container {
    flex: 1;
    min-width: 200px;
}

.dropdown-label {
    display: block;
    margin-bottom: 5px;
    font-weight: 600;
    color: #34495e;
}

.dropdown {
    width: 100%;
}

.metrics-container {
    display: flex;
    gap: 20px;
    margin-bottom: 30px;
    flex-wrap: wrap;
}

.metric-card {
    flex: 1;
    min-width: 200px;
    background: #f8f9fa;
    border-radius: 8px;
    padding: 20px;
    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
}

.graphs-row {
    display: flex;
    gap: 20px;
    margin-bottom: 20px;
}

.graph {
    flex: 1;
    min-width: 0;
    border-radius: 8px;
    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    background: white;
}

.table-header {
    margin: 30px 0 15px 0;
    color: #2c3e50;
}

.table-container {
    overflow-x: auto;
    margin-bottom: 50px;
}

/* Responsive adjustments */
@media (max-width: 768px) {
    .filters-row {
        flex-direction: column;
    }

    .graphs-row {
        flex-direction: column;
    }
}
"""

app.index_string = f'''
<!DOCTYPE html>
<html>
    <head>
        {{%metas%}}
        <title>Claims Dashboard</title>
        {{%favicon%}}
        {{%css%}}
        <style>{styles}</style>
    </head>
    <body>
        {{%app_entry%}}
        <footer>
            {{%config%}}
            {{%scripts%}}
            {{%renderer%}}
        </footer>
    </body>
</html>
'''
"""))

# Code cell: Callbacks
cells.append(v4.new_code_cell("""\
# Callback to filter data based on inputs
@app.callback(
    Output('filtered-data-store', 'data'),
    [Input('stage-dropdown', 'value'),
     Input('insurer-dropdown', 'value'),
     Input('type-dropdown', 'value'),
     Input('year-dropdown', 'value')]
)
def update_filtered_data(stage, insurer, ctype, year):
    dff = df.copy()
    dff = dff[dff['claim_stage'] == stage]

    if insurer != 'All':
        dff = dff[dff['insurer'] == insurer]
    if ctype != 'All':
        dff = dff[dff['claim_type'] == ctype]
    if year != 'All':
        dff = dff[dff['year'] == year]

    return dff.to_dict('records')

# Callback to update metrics
@app.callback(
    [Output('total-claims', 'children'),
     Output('total-amount', 'children'),
     Output('avg-amount', 'children')],
    [Input('filtered-data-store', 'data')]
)
def update_metrics(data):
    dff = pd.DataFrame(data)
    if dff.empty:
        return ("Total Claims: 0", "Total Amount: $0.00", "Avg Amount: $0.00")

    total_claims = len(dff)
    total_amount = dff['claim_amount'].sum()
    avg_amount = dff['claim_amount'].mean()

    return (
        f"Total Claims: {total_claims:,}",
        f"Total Amount: ${total_amount:,.2f}",
        f"Avg Amount: ${avg_amount:,.2f}"
    )

# Callback for claims by year chart
@app.callback(
    Output('claims-by-year', 'figure'),
    [Input('filtered-data-store', 'data')]
)
def update_claims_by_year(data):
    dff = pd.DataFrame(data)
    if dff.empty:
        return px.bar(title='No data available for selected filters')

    grouped = dff.groupby('year').agg(
        Claims=('claim_amount', 'count'),
        Total_Amount=('claim_amount', 'sum')
    ).reset_index()

    grouped['Total_Amount_M'] = grouped['Total_Amount'] / 1_000_000

    fig = px.bar(grouped, x='year', y='Claims',
                 title='Claims Count and Amount per Year',
                 labels={'Claims': 'Number of Claims', 'year': 'Year'})

    fig.add_scatter(x=grouped['year'], y=grouped['Total_Amount_M'],
                   mode='lines+markers', name='Amount (Millions $)',
                   yaxis='y2', line=dict(color='orange'))

    fig.update_layout(
        yaxis=dict(title='Number of Claims'),
        yaxis2=dict(title='Claim Amount (Millions $)',
                   overlaying='y', side='right'),
        hovermode='x unified',
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)'
    )

    return fig

# Callback for amount distribution chart
@app.callback(
    Output('amount-distribution', 'figure'),
    [Input('filtered-data-store', 'data')]
)
def update_amount_distribution(data):
    dff = pd.DataFrame(data)
    if dff.empty:
        return px.box(title='No data available for selected filters')

    return px.box(dff, y='claim_amount',
                 title='Claim Amount Distribution',
                 labels={'claim_amount': 'Claim Amount ($)'})

# Callback for claims by insurer chart
@app.callback(
    Output('claims-by-insurer', 'figure'),
    [Input('filtered-data-store', 'data')]
)
def update_claims_by_insurer(data):
    dff = pd.DataFrame(data)
    if dff.empty:
        return px.bar(title='No data available for selected filters')

    return px.bar(dff['insurer'].value_counts().reset_index(),
                 x='index', y='insurer',
                 title='Claims by Insurer',
                 labels={'index': 'Insurer', 'insurer': 'Number of Claims'})

# Callback for claims by type chart
@app.callback(
    Output('claims-by-type', 'figure'),
    [Input('filtered-data-store', 'data')]
)
def update_claims_by_type(data):
    dff = pd.DataFrame(data)
    if dff.empty:
        return px.bar(title='No data available for selected filters')

    return px.pie(dff, names='claim_type',
                 title='Claims by Type',
                 hole=0.4)

# Callback for data table
@app.callback(
    Output('data-table', 'children'),
    [Input('filtered-data-store', 'data')]
)
def update_data_table(data):
    dff = pd.DataFrame(data)
    if dff.empty:
        return html.Div("No data available for selected filters", className="no-data")

    return dash_table.DataTable(
        data=dff.to_dict('records'),
        columns=[{'name': col, 'id': col} for col in dff.columns],
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={
            'textAlign': 'left',
            'padding': '8px',
            'minWidth': '100px'
        },
        style_header={
            'backgroundColor': '#f8f9fa',
            'fontWeight': 'bold'
        },
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ]
    )
"""))

# Code cell: Run app
cells.append(v4.new_code_cell("""\
if __name__ == '__main__':
    if IN_COLAB:
        app.run_server()
    else:
        app.run_server(debug=DEBUG, port=PORT)
"""))

# Create notebook
nb = v4.new_notebook(cells=cells)
output_path = Path("ClaimsDashboard.ipynb")
with open(output_path, "w", encoding="utf-8") as f:
    write(nb, f)

print(f"Notebook created at: {output_path.absolute()}")